In [1]:
!cd .. && make clean

rm -rf build


In [2]:
!cd .. && make cia-unpack

Running unpacking tool.
bin/unpack.sh cias/bd.cia build/dev
~/projects/bravely-default-mod/build/dev ~/projects/bravely-default-mod
[1] Extracting CIA with ctrtool to build/dev
CiaHeader:
|- HeaderSize:      0x2020
|- Type:            Normal (0x0000)
|- FormatVersion:   Cia (0x0000)
|- CertificateSize: 0xa00
|- TicketSize:      0x350
|- TitleMetaSize:   0xb64
|- FooterSize:      0x3ac0
|- ContentSize:     0xcef27000
\- EnabledContent:
   |- 0x0000
   \- 0x0001
Certificate Chain:
|- Certificate 0:
|  |- DigitalSignature: 
|  |  |- SigType:    RSA-4096-SHA256 (0x10003)
|  |  |- Issuer:     Root
|  |  \- Signature:  704138EF...61CB4C2E
|  |- Subject:       CA00000003
|  \- PublicKey:     RSA-2048 (0x1)
|     |- m:          B279C9E2...EBEED379
|     \- e:          00010001
|- Certificate 1:
|  |- DigitalSignature: 
|  |  |- SigType:    RSA-2048-SHA256 (0x10004)
|  |  |- Issuer:     Root-CA00000003
|  |  \- Signature:  919EBE46...08847982
|  |- Subject:       XS0000000c
|  \- PublicKey:    

In [7]:
!cd .. && MAMBA_ROOT_PREFIX=/home/struktured/micromamba make crowd-unpack

micromamba run -n "bd-dev" bin/crowd.sh -r build/dev -o build/crowd-dev-unpacked -g BD unpack
Output directory: build/crowd-dev-unpacked
Output directory: /home/struktured/projects/bravely-default-mod/build/crowd-dev-unpacked
Loaded content.0000.cxi.0000.00000000
Loaded content.0000.cxi.0001.00000001
Loaded cxi/exefs.bin
Loaded cxi/header.bin
Loaded cxi/plain.bin
Loaded cxi/romfs.bin
Loaded cxi/exheader.bin
Loaded cxi/romfs_dir/Common_fr/DReportTable/crowd.fs
Dumping spreadsheet cxi/romfs_dir/Common_fr/DReportTable/crowd.fs
   building cxi/romfs_dir/Common_fr/DReportTable/DarkKnightNoteData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DarkKnightNoteIndex.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DTableBonusData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DTableItemData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DTableMonsterData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/GenomData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/Genom

In [8]:
import sys
import pandas as pd
sys.path.insert(0, "../")

import arcanist
from arcanist.crowd import loader
import importlib
importlib.reload(arcanist)
importlib.reload(loader)

CROWD_DATA="../build/crowd-dev-unpacked"
PAT = "BuffLimit"

data = loader.annotate(crowd_data=CROWD_DATA, pat=PAT)


check col DarkKnightNoteData.btb/Label
check col DarkKnightNoteData.btb/Label.1
check col DarkKnightNoteData.btb/Label.2
check col DarkKnightNoteData.btb/Text
check col DarkKnightNoteData.btb/Text.1
check col DarkKnightNoteData.btb/Text.2
check col DarkKnightNoteData.btb/Unnamed: 6
check col DarkKnightNoteData.btb/Text Pntr
check col DarkKnightNoteData.btb/Text Pntr.1
check col DarkKnightNoteData.btb/Unnamed: 9
check col DarkKnightNoteData.btb/Text Pntr.2
check col DarkKnightNoteData.btb/Label Pntr
check col DarkKnightNoteData.btb/Label Pntr.1
check col DarkKnightNoteData.btb/Label Pntr.2
check col DarkKnightNoteIndex.btb/Text
check col DarkKnightNoteIndex.btb/Unnamed: 1
check col DarkKnightNoteIndex.btb/Text Pntr
check col DarkKnightNoteIndex.btb/Unnamed: 3
check col DarkKnightNoteIndex.btb/Unnamed: 4
check col DTableBonusData.btb/Label
check col DTableBonusData.btb/Label.1
check col DTableBonusData.btb/Text
check col DTableBonusData.btb/Text.1
check col DTableBonusData.btb/Text.2
che

In [9]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

files : list[str] = []
schemas :list[str]= []
for k,v in data.items():    
    for schema in v.keys():
        files.append(k)
        schemas.append(schema)

files_schemas = pd.DataFrame({'files':  files, 'schemas': schemas})

pd.DataFrame({'unique': files_schemas[files_schemas['files'].str.contains('Common_en')].files.unique()})

,unique
0,Common_en/Battle


In [10]:
battle_data = data['Common_en/Battle']
battle_data['BuffLimit.btb']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,150,150,150,150,150,150,1000,150,700,700,150,200,200
1,75,75,75,75,75,75,75,75,10,10,75,75,75


In [11]:

from arcanist.crowd.loader import CrowdSchema, CrowdSchemaOverrides


battle_data = ["Common/Battle", "Common_en/Battle"]

overrides : list[CrowdSchemaOverrides] = []
for path in battle_data:
    overrides.append(CrowdSchemaOverrides.builder(path).add(\
              sheet="BuffLimit.btb", 
              index={0: 'MaxPercent', 1:'MinPercent'}, 
              columns={0:'PhysicalAttack', 1: 'MagicAttack', 2: 'PhysicalDefense', 3: 'MagicDefense', 4: 'CriticalHit', 5: 'Speed', 6: 'HitCount'}
    ).build())

print(overrides)
annot=loader.annotate(crowd_data=data, overrides=overrides)


[CrowdSchemaOverrides(path='Common/Battle', schemas=[CrowdSchema(sheet='BuffLimit.btb', index={0: 'MaxPercent', 1: 'MinPercent'}, columns={0: 'PhysicalAttack', 1: 'MagicAttack', 2: 'PhysicalDefense', 3: 'MagicDefense', 4: 'CriticalHit', 5: 'Speed', 6: 'HitCount'})]), CrowdSchemaOverrides(path='Common_en/Battle', schemas=[CrowdSchema(sheet='BuffLimit.btb', index={0: 'MaxPercent', 1: 'MinPercent'}, columns={0: 'PhysicalAttack', 1: 'MagicAttack', 2: 'PhysicalDefense', 3: 'MagicDefense', 4: 'CriticalHit', 5: 'Speed', 6: 'HitCount'})])]
Renamed column Unnamed: 0 to PhysicalAttack in sheet BuffLimit.btb.
Renamed column Unnamed: 1 to MagicAttack in sheet BuffLimit.btb.
Renamed column Unnamed: 2 to PhysicalDefense in sheet BuffLimit.btb.
Renamed column Unnamed: 3 to MagicDefense in sheet BuffLimit.btb.
Renamed column Unnamed: 4 to CriticalHit in sheet BuffLimit.btb.
Renamed column Unnamed: 5 to Speed in sheet BuffLimit.btb.
Renamed column Unnamed: 6 to HitCount in sheet BuffLimit.btb.
Renamed 

In [12]:

for path in battle_data:
    buff_limits = annot[path]['BuffLimit.btb']
    buff_limits.loc['MaxPercent', 'PhysicalAttack'] = buff_limits['PhysicalAttack']['MaxPercent']*2 + 1
 #   buff_limits.loc['MaxPercent', 'MagicAttack'] = buff_limits['MagicAttack']['MaxPercent']*2 + 2
 #   buff_limits.loc['MaxPercent', 'PhysicalDefense'] = buff_limits['PhysicalDefense']['MaxPercent']*2 + 3
 #   buff_limits.loc['MaxPercent', 'MagicDefense'] = buff_limits['MagicDefense']['MaxPercent']*2 + 4
#    buff_limits.loc['MaxPercent', 'CriticalHit'] = buff_limits['CriticalHit']['MaxPercent']*2 + 5
#    buff_limits.loc['MaxPercent', 'Speed'] = buff_limits['Speed']['MaxPercent']*2 + 6
#    buff_limits.loc['MaxPercent', 'HitCount'] = buff_limits['HitCount']['MaxPercent']*2 + 7



In [13]:
buff_limits

,PhysicalAttack,MagicAttack,PhysicalDefense,MagicDefense,CriticalHit,Speed,HitCount,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
MaxPercent,301,150,150,150,150,150,1000,150,700,700,150,200,200
MinPercent,75,75,75,75,75,75,75,75,10,10,75,75,75


In [16]:
importlib.reload(loader)
loader.save(annot, root_dir=CROWD_DATA)

Saved crowd data to ../build/crowd-dev-unpacked/cxi/romfs_dir/Common_en/Battle/BuffLimit.btb.parquet
Saved crowd data to ../build/crowd-dev-unpacked/cxi/romfs_dir/Common_de/Battle/BuffLimit.btb.parquet
Saved crowd data to ../build/crowd-dev-unpacked/cxi/romfs_dir/Common_es/Battle/BuffLimit.btb.parquet
Saved crowd data to ../build/crowd-dev-unpacked/cxi/romfs_dir/Common/Battle/BuffLimit.btb.parquet
Saved crowd data to ../build/crowd-dev-unpacked/cxi/romfs_dir/Common_it/Battle/BuffLimit.btb.parquet
Saved crowd data to ../build/crowd-dev-unpacked/cxi/romfs_dir/Common_fr/Battle/BuffLimit.btb.parquet


In [17]:
!cd .. && MAMBA_ROOT_PREFIX=/home/struktured/micromamba make crowd-pack

micromamba run -n "bd-dev" bin/crowd.sh -r build/crowd-dev-unpacked -o build/crowd-dev-packed -g BD pack
Output directory: build/crowd-dev-packed
Output directory: /home/struktured/projects/bravely-default-mod/build/crowd-dev-packed
Packing....
CHDIR: /home/struktured/projects/bravely-default-mod/build/crowd-dev-unpacked
INFO:arcanist.crowd.core.classes:Loading spreadsheet cxi/romfs_dir/Common_fr/DReportTable/crowd.parquet...
INFO:arcanist.crowd.core.classes:Loading spreadsheet cxi/romfs_dir/Common_fr/PartyChat/crowd.parquet...
INFO:arcanist.crowd.core.classes:Loading spreadsheet cxi/romfs_dir/Common_fr/Paramater/crowd.parquet...
INFO:arcanist.crowd.core.classes:Loading spreadsheet cxi/romfs_dir/Common_fr/Shop/crowd.parquet...
INFO:arcanist.crowd.core.classes:Loading spreadsheet cxi/romfs_dir/Common_fr/MiniMapIconTable/crowd.parquet...
INFO:arcanist.crowd.core.classes:Loading spreadsheet cxi/romfs_dir/Common_fr/TreasureTable/crowd.parquet...
INFO:arcanist.crowd.core.classes:Loading spr

In [ ]:
!cd .. && make deploy-romfs

QUALIFIER=dev USER=carm BUILD_DIR=build CITRA_DIR=/data/citra/citra-emu bin/deploy_romfs.sh
+ VERSION=00040000000FC500
+ QUALIFIER=dev
+ ROMFS_DIR=build/crowd-dev-packed/00040000000FC500/romfs
+ CITRA_DIR=/data/citra/citra-emu
++ date +%Y%m%d_%H%M%S
+ BACKUP=20250802_235139
+ BACKUP_DIR=build/dev/backup
+ FROM=
+ TO=/data/citra/citra-emu/load/mods/00040000000FC500/romfs
+ mkdir -p build/dev/backup
+ [[ -z '' ]]
+ FROM=build/crowd-dev-packed/00040000000FC500/romfs
+ echo Using default from build/crowd-dev-packed/00040000000FC500/romfs
Using default from build/crowd-dev-packed/00040000000FC500/romfs
+ set +e
+ mkdir -p /data/citra/citra-emu/load/mods/00040000000FC500/romfs
+ set -e
+ echo 'Backing up original romfs'
Backing up original romfs
+ cp -r -v /data/citra/citra-emu/load/mods/00040000000FC500/romfs build/dev/backup/romfs_dir.20250802_235139.bak
'/data/citra/citra-emu/load/mods/00040000000FC500/romfs' -> 'build/dev/backup/romfs_dir.20250802_235139.bak'
'/data/citra/citra-emu/load/

In [ ]:

!cd .. && make deploy-code

QUALIFIER=dev USER=carm BUILD_DIR=build CITRA_DIR=/data/citra/citra-emu bin/deploy_code.sh 
+ QUALIFIER=dev
+ EXEFS_DIR=build/crowd-dev-unpacked/cxi/exefs_dir
+ CITRA_DIR=/data/citra/citra-emu
++ date +%Y%m%d_%H%M%S
+ BACKUP=20250802_235139
+ VERSION=00040000000FC500
+ BACKUP_DIR=build/dev/backup
+ FROM=
+ TO=/data/citra/citra-emu/load/mods/00040000000FC500/exefs/code.bin
+ mkdir -p build/dev/backup
+ [[ -z '' ]]
+ FROM=build/crowd-dev-unpacked/cxi/exefs_dir/code.bin
+ echo Using default from build/crowd-dev-unpacked/cxi/exefs_dir/code.bin
Using default from build/crowd-dev-unpacked/cxi/exefs_dir/code.bin
+ echo 'Backing up original code.bin'
Backing up original code.bin
+ cp -v /data/citra/citra-emu/load/mods/00040000000FC500/exefs/code.bin build/dev/backup/code.bin.20250802_235139.bak
'/data/citra/citra-emu/load/mods/00040000000FC500/exefs/code.bin' -> 'build/dev/backup/code.bin.20250802_235139.bak'
+ echo 'Deploying new code.bin'
Deploying new code.bin
+ cp -v build/crowd-dev-unpack